Import necessary libraries

In [ ]:
import pickle

from Robotic_Arm.rm_robot_interface import *


Create a RoboticArm instance and connect to the robotic arm

In [ ]:
robot = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
handle = robot.rm_create_robot_arm("192.168.110.118", 8080)
print("Arm ID：", handle.id)


basic operations to check the connection and arm state

In [ ]:
software_info = robot.rm_get_arm_software_info()
if software_info[0] == 0:
    print("\n================== Arm Software Information ==================")
    print("Arm Model: ", software_info[1]['product_version'])
    print("Algorithm Library Version: ", software_info[1]['algorithm_info']['version'])
    print("Control Layer Software Version: ", software_info[1]['ctrl_info']['version'])
    print("Dynamics Version: ", software_info[1]['dynamic_info']['model_version'])
    print("Planning Layer Software Version: ", software_info[1]['plan_info']['version'])
    print("==============================================================\n")
else:
    print("\nFailed to get arm software information, Error code: ", software_info[0], "\n")


Check the current work frame and tool frame

In [ ]:
print(robot.rm_change_work_frame("World"))

print("Current Arm State: ")
print(robot.rm_get_current_arm_state())

print("All Work Frames: ")
print(robot.rm_get_total_work_frame())

print("Current Work Frame: ")
print(robot.rm_get_current_work_frame())

print("All Tool Frames: ")
print(robot.rm_get_total_tool_frame())

print("Current Tool Frame: ")
print(robot.rm_get_current_tool_frame())

Check the safety settings and collision stage

In [ ]:

print(robot.rm_set_collision_state(8))
print(robot.rm_get_collision_stage())


In [ ]:

# print(robot.rm_set_arm_max_line_speed(0.05))
print(robot.rm_get_arm_max_line_speed())

# print(robot.rm_set_arm_max_line_acc(0.1))
print(robot.rm_get_arm_max_line_acc())


In [ ]:

# print(robot.rm_set_arm_max_angular_speed(0.2))
print(robot.rm_get_arm_max_angular_speed())

# print(robot.rm_set_arm_max_angular_acc(1))
print(robot.rm_get_arm_max_angular_acc())


In [ ]:
# default parameters:
# End-effector linear speed: 0.25 m/s    End-effector linear acceleration
# End-effector angular speed: 0.6 rad/s  End-effector angular acceleration: 4 rad/s²

print(robot.rm_set_arm_tcp_init())


IK and FK check

In [ ]:
print(robot.rm_movej([1.8220000267028809, 22.739999771118164, 48.632999420166016, 176.00900268554688, -107.83200073242188, 211.63499450683594], 20, 0, 0, True))
print("Current Arm State: ", robot.rm_get_current_arm_state())

In [ ]:
arm_model = rm_robot_arm_model_e.RM_MODEL_RM_65_E  # RM_65机械臂
force_type = rm_force_type_e.RM_MODEL_RM_B_E  # 标准版
algo_handle = Algo(arm_model, force_type)
algo_handle.rm_algo_set_redundant_parameter_traversal_mode(False)

curr_state = robot.rm_get_current_arm_state()
print("Current Arm State: ", curr_state)

curr_pose = curr_state[1]['pose']


In [ ]:
curr_state = robot.rm_get_current_arm_state()
curr_pose = curr_state[1]['pose']


target_pose = curr_pose.copy()

# x == -0.3
target_pose[0] = -0.3
# target_pose[0] = -0.3 + 0.05

# y in [-0.2, 0.2]
target_pose[1] = -0.1
# target_pose[1] = -0.084

# z in [0.2, 0.35]
target_pose[2] = 0.1
# target_pose[2] = 0.315

# forehand == [-3.093, 0.047, -0.54]
target_pose[3:] = [-3.093, 0.047, -0.54]

# backhand == [3.073, -0.029, 2.673]
# target_pose[3:] = [3.073, -0.029, 2.673]


print("Target Pose: ", target_pose)

params = rm_inverse_kinematics_params_t(
    q_in=curr_state[1]['joint'],
    q_pose=target_pose,
    flag=1,
)

q_out = algo_handle.rm_algo_inverse_kinematics(params)
print("IK result: ", q_out)

print(robot.rm_movej(q_out[1], 20, 0, 0, True))

target_joint_angles = q_out[1]
print("Target Joint Angles: ", target_joint_angles)
target_joint_angles[4] += 15
# target_joint_angles[4] -= 30

# print(robot.rm_movej(target_joint_angles, 20, 0, 0, True))


print("Current Arm State: ", robot.rm_get_current_arm_state())

Close the connection to the robotic arm

In [ ]:
robot.rm_delete_robot_arm()